In [1]:
import numpy as np
import torch as th
from torch import nn as nn
import torch.nn.functional as F
from torch import tensor
    

In [2]:
model_path = 'tmp/best_model/best_model'

In [68]:
class ConvNet(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(7, 32, kernel_size=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=1)
        self.fc3 = nn.Linear(2112, 512)
        self.shared1 = nn.Linear(512, 64)
        self.policy1 = nn.Linear(64, 32)
        self.policy2 = nn.Linear(32, 16)
        self.action = nn.Linear(16, 4)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = nn.Flatten()(x)
        x = F.relu(self.fc3(x))
        x = F.relu(self.shared1(x))
        x = F.relu(self.policy1(x))
        x = F.relu(self.policy2(x))
        x = self.action(x)
        x = x.argmax()
        return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.shared1 = th.nn.Sequential()
        self.policy1 = nn.Linear(231, 64)
        self.policy2 = nn.Linear(64, 64)
        self.action = nn.Linear(64, 4)

    def forward(self, x):
        x = nn.Flatten(start_dim=1, end_dim=-1)(x)
        x = F.tanh(self.policy1(x))
        x = F.tanh(self.policy2(x))
        x = self.action(x)
        x = x.argmax()
        return x


mlp_extractor.policy_net.0.weight', 'mlp_extractor.policy_net.0.bias', 'mlp_extractor.policy_net.2.weight', 'mlp_extractor.policy_net.2.bias', 'mlp_extractor.value_net.0.weight', 'mlp_extractor.value_net.0.bias', 'mlp_extractor.value_net.2.weight', 'mlp_extractor.value_net.2.bias', 'action_net.weight', 'action_net.bias', 'value_net.weight', 'value_net.bias'

ctorCriticPolicy(  
  (features_extractor): FlattenExtractor(  
    (flatten): Flatten(start_dim=1, end_dim=-1)  
  )  
  (mlp_extractor): MlpExtractor(  
    (shared_net): Sequential()  
    (policy_net): Sequential(  
      (0): Linear(in_features=231, out_features=64, bias=True)  
      (1): Tanh()  
      (2): Linear(in_features=64, out_features=64, bias=True)  
      (3): Tanh()  
    )  
    (value_net): Sequential(  
      (0): Linear(in_features=231, out_features=64, bias=True)  
      (1): Tanh() 
      (2): Linear(in_features=64, out_features=64, bias=True)  
      (3): Tanh()  
    )  
  )  
  (action_net): Linear(in_features=64, out_features=4, bias=True)  
  (value_net): Linear(in_features=64, out_features=1, bias=True)  
)  

In [69]:
model = Net()

In [99]:
model = th.load('best_model/policy.pth')

In [100]:
model

OrderedDict([('mlp_extractor.policy_net.0.weight',
              tensor([[-0.1451, -0.0613, -0.1015,  ..., -0.1770,  0.0284,  0.0035],
                      [-0.0029, -0.0002, -0.0300,  ...,  0.1078,  0.0113, -0.0608],
                      [ 0.0983,  0.0087,  0.0754,  ...,  0.0177, -0.0246, -0.0696],
                      ...,
                      [ 0.0118,  0.1192, -0.0972,  ...,  0.1197, -0.0740,  0.0981],
                      [ 0.0280, -0.0479,  0.1426,  ...,  0.0137, -0.0828, -0.1245],
                      [-0.1264,  0.1727, -0.0023,  ..., -0.2118, -0.0293, -0.0548]],
                     device='cuda:0')),
             ('mlp_extractor.policy_net.0.bias',
              tensor([ 1.0918e-04,  3.1626e-05,  5.7694e-05, -4.9769e-06,  1.0920e-05,
                       2.0709e-05, -4.7005e-06,  5.1650e-05,  1.9498e-05,  1.1115e-05,
                      -4.3169e-05, -1.2275e-05, -3.6334e-05, -2.7966e-05,  3.6487e-05,
                      -7.9229e-06, -4.3484e-05,  1.3143e-05,  4.573

In [7]:
def get_geese_observation(state):
    """
    Given a particular geese, does some processing and returns a geese specific observation. 
    Unfortunately specific to the geese environment for now.
    Encoding as follows: 
    2: enemy snake head
    1: enemy snake body
    11: own head
    12: own body
    100: food
    """

    if type(state) is list:
        state = state[0].observation
    else:

        state = Observation(state)

    
    agent = state.index
    rows = 7
    columns = 11
    game_board_self = np.zeros(rows*columns, None)
    game_board_enemy = np.zeros(rows*columns, None)
    game_board_food = np.zeros(rows*columns, None)


    for i, geese in enumerate(state.geese):
        identify=0
        if i==agent:
            identify=100
            for j, cell in enumerate(geese):
                if j == 0:
                    game_board_self[cell] = identify+1
                else:
                    game_board_self[cell] = identify+2
        else:
            identify=-100
            for j, cell in enumerate(geese):
                if j == 0:
                    game_board_enemy[cell] = identify+1
                else:
                    game_board_enemy[cell] = identify+2

    for food in state.food:
        game_board_food[food] = 1000
    game_board_self = game_board_self.reshape([rows, columns])
    game_board_enemy = game_board_enemy.reshape([rows, columns])
    game_board_food = game_board_food.reshape([rows, columns])

    head = get_geese_coord(game_board_self)[0]


    game_board_self = np.roll(game_board_self, 5-head[1], axis=1)
    game_board_self = np.roll(game_board_self, 3-head[0], axis=0)
    game_board_enemy = np.roll(game_board_enemy, 5-head[1], axis=1)
    game_board_enemy = np.roll(game_board_enemy, 3-head[0], axis=0)
    game_board_food = np.roll(game_board_food, 5-head[1], axis=1)
    game_board_food = np.roll(game_board_food, 3-head[0], axis=0)

    #game_board = game_board.reshape((game_board.shape[0], game_board.shape[1], 1))
    game_board = np.dstack((game_board_self, game_board_enemy, game_board_food))
    return game_board

def get_geese_coord(board):
    return get_coord_from_np_grid(board, 101)

def get_food_coord(board):
    return get_coord_from_np_grid(board, 1000)

def get_enemy_geese_head_coord(board):
    return get_coord_from_np_grid(board, -99)


def get_coord_from_np_grid(grid, value):
    coords = []
    for i in range(0, len(np.where(grid==value)[0])):
        coords.append((np.where(grid==value)[0][i], np.where(grid==value)[1][i]))
    return coords


In [8]:
state_dict = {
    'conv1.weight': state_dict['features_extractor.conv1.weight'],
    'conv1.bias': state_dict['features_extractor.conv1.bias'],
    'conv2.weight': state_dict['features_extractor.conv2.weight'],
    'conv2.bias': state_dict['features_extractor.conv2.bias'],
    'fc3.weight': state_dict['features_extractor.fc3.weight'],
    'fc3.bias': state_dict['features_extractor.fc3.bias'],

    'shared1.weight': state_dict['mlp_extractor.shared_net.0.weight'],
    'shared1.bias': state_dict['mlp_extractor.shared_net.0.bias'],

    'policy1.weight': state_dict['mlp_extractor.policy_net.0.weight'],
    'policy1.bias': state_dict['mlp_extractor.policy_net.0.bias'],
    'policy2.weight': state_dict['mlp_extractor.policy_net.2.weight'],
    'policy2.bias': state_dict['mlp_extractor.policy_net.2.bias'],

    'action.weight': state_dict['action_net.weight'],
    'action.bias': state_dict['action_net.bias'],
}

NameError: name 'state_dict' is not defined

In [131]:
def agent(obs, config):
    model = Net()
    model = model.float()
    state_dict = th.load('best_pytorch_model')
    state_dict = {
        'policy1.weight': state_dict['mlp_extractor.policy_net.0.weight'],
        'policy1.bias': state_dict['mlp_extractor.policy_net.0.bias'],
        'policy2.weight': state_dict['mlp_extractor.policy_net.2.weight'],
        'policy2.bias': state_dict['mlp_extractor.policy_net.2.bias'],
        'action.weight': state_dict['action_net.weight'],
        'action.bias': state_dict['action_net.bias'],
    }
    model.load_state_dict(state_dict)
    model.eval()
    obs = tensor(get_geese_observation(obs)).reshape(1, 7, 11, 3).float()
    action = model(obs)
    print(action)
    return int(action)

In [10]:
from GeeseGymWrapper import HungryGeeseKaggle

Loading environment football failed: No module named 'gfootball'


In [80]:
env = HungryGeeseKaggle()

In [86]:
env.env.reset(num_agents=4)

[{'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60,
   'step': 0,
   'geese': [[42], [39], [51], [41]],
   'food': [29, 13],
   'index': 0},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 1},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 2},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 3},
  'status': 'ACTIVE'}]

In [132]:
obs = env.env.state

AttributeError: 'HungryGeeseKaggle' object has no attribute 'state'

In [88]:
obs

[{'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60,
   'step': 0,
   'geese': [[42], [39], [51], [41]],
   'food': [29, 13],
   'index': 0},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 1},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 2},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 3},
  'status': 'ACTIVE'}]

In [14]:
config = {}

In [134]:
agent(obs, config)

tensor(2)


2

In [ ]:
import os
os.listdir()

In [ ]:
path = os.getcwd()

In [ ]:
path

In [127]:
# Check that if I load the same model from stable baselines I get the same output
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor

log_dir = "tmp/"
env = HungryGeeseKaggle()
env = Monitor(env, log_dir)
model = PPO('MlpPolicy', env, verbose=1)



Using cuda device
Wrapping the env in a DummyVecEnv.


In [128]:
model = model.load('best_model.zip')

In [129]:
obs

[{'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60,
   'step': 0,
   'geese': [[42], [39], [51], [41]],
   'food': [29, 13],
   'index': 0},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 1},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 2},
  'status': 'ACTIVE'},
 {'action': 'NORTH',
  'reward': 0,
  'info': {},
  'observation': {'remainingOverageTime': 60, 'index': 3},
  'status': 'ACTIVE'}]

In [130]:
model.policy.predict(get_geese_observation(obs), deterministic=True)

(2, None)

In [121]:
model.policy.state_dict()

OrderedDict([('mlp_extractor.policy_net.0.weight',
              tensor([[-0.1451, -0.0613, -0.1015,  ..., -0.1770,  0.0284,  0.0035],
                      [-0.0029, -0.0002, -0.0300,  ...,  0.1078,  0.0113, -0.0608],
                      [ 0.0983,  0.0087,  0.0754,  ...,  0.0177, -0.0246, -0.0696],
                      ...,
                      [ 0.0118,  0.1192, -0.0972,  ...,  0.1197, -0.0740,  0.0981],
                      [ 0.0280, -0.0479,  0.1426,  ...,  0.0137, -0.0828, -0.1245],
                      [-0.1264,  0.1727, -0.0023,  ..., -0.2118, -0.0293, -0.0548]],
                     device='cuda:0')),
             ('mlp_extractor.policy_net.0.bias',
              tensor([ 1.0918e-04,  3.1626e-05,  5.7694e-05, -4.9769e-06,  1.0920e-05,
                       2.0709e-05, -4.7005e-06,  5.1650e-05,  1.9498e-05,  1.1115e-05,
                      -4.3169e-05, -1.2275e-05, -3.6334e-05, -2.7966e-05,  3.6487e-05,
                      -7.9229e-06, -4.3484e-05,  1.3143e-05,  4.573